## Fuse radiomics features across all subjects and normalize by height squared


In [64]:
import pandas as pd
import os

In [67]:
nifti_path = f"{PROJECT_DIR}/data/data/whole_body/nifti"
eids_nifti = os.listdir(nifti_path)
eids_nifti = [int(eid) for eid in eids_nifti if eid.isdigit()]

radiomics_revised_path = "/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/radiomics_revised"
radiomics_df = pd.read_csv(radiomics_revised_path + "/1001399/radiomics_features_wat.csv")


In [68]:
import pandas as pd
import os

# Define the base directory
base_dir = "/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/radiomics_revised/"

# List all directories (each representing an eid)
eids = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

# Initialize an empty list to collect dataframes
dfs = []
count = 0
# Loop through each eid directory
for eid in eids:
    eid_path = os.path.join(base_dir, eid)
    print(f"Processing {eid} ({count}/{len(eids)})")
    
    # Define paths to radiomics files
    wat_file = os.path.join(eid_path, "radiomics_features_wat.csv")
    fat_file = os.path.join(eid_path, "radiomics_features_fat.csv")
    
    # Load the files and add suffixes
    if os.path.exists(wat_file):
        df_wat = pd.read_csv(wat_file)
        df_wat = df_wat.add_suffix('_wat')
        df_wat['eid'] = eid  # Add eid column
    else:
        df_wat = pd.DataFrame()  # Empty if file is missing
        
    if os.path.exists(fat_file):
        df_fat = pd.read_csv(fat_file)
        df_fat = df_fat.add_suffix('_fat')
        df_fat['eid'] = eid  # Add eid column
    else:
        df_fat = pd.DataFrame()  # Empty if file is missing

    # Merge the wat and fat dataframes on eid, if both are non-empty
    if not df_wat.empty and not df_fat.empty:
        combined_df = pd.merge(df_wat, df_fat, on='eid', how='outer')
    elif not df_wat.empty:
        combined_df = df_wat
    else:
        combined_df = df_fat

    # Append the combined dataframe to the list
    dfs.append(combined_df)
    count += 1


# Concatenate all dataframes
result_df = pd.concat(dfs, ignore_index=True)



Processing 5191418 (0/5762)
Processing 2291873 (1/5762)
Processing 2582362 (2/5762)
Processing 2616817 (3/5762)
Processing 3187770 (4/5762)
Processing 1024387 (5/5762)
Processing 5498166 (6/5762)
Processing 4334290 (7/5762)
Processing 5944002 (8/5762)
Processing 4884093 (9/5762)
Processing 2270077 (10/5762)
Processing 1250622 (11/5762)
Processing 3519007 (12/5762)
Processing 2967043 (13/5762)
Processing 5699374 (14/5762)
Processing 5869410 (15/5762)
Processing 2325278 (16/5762)
Processing 3048783 (17/5762)
Processing 5610908 (18/5762)
Processing 2339146 (19/5762)
Processing 3552383 (20/5762)
Processing 1405281 (21/5762)
Processing 4511810 (22/5762)
Processing 5977575 (23/5762)
Processing 2216650 (24/5762)
Processing 3910295 (25/5762)
Processing 5508966 (26/5762)
Processing 1632560 (27/5762)
Processing 2982268 (28/5762)
Processing 2294914 (29/5762)
Processing 5440368 (30/5762)
Processing 4521426 (31/5762)
Processing 2575362 (32/5762)
Processing 3844788 (33/5762)
Processing 5962910 (34/5

In [69]:
result_df.to_csv("/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/radiomics_revised.csv", index=False)

In [3]:
result_df = pd.read_csv("/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/radiomics_revised.csv")

/tmp/ipykernel_1107844/2164037943.py:1: DtypeWarning: Columns (15199,15200,15201,15202,15203,15204,15205,15206,15207,15208,15209,15210,15211,15212,15213,15214,15215,15216,15217,15218,15219,15220,15221,15222,15223,15224,15225,15226,15227,15228,15229,15230,15231,15232,15233,15234,15235,15236,15237,15238,15239,15240,15241,15242,15243,15244,15245,15246,15247,15248,15249,15250,15251,15252,15253,15254,15255,15256,15257,15258,15259,15260) have mixed types. Specify dtype option on import or set low_memory=False.
  result_df = pd.read_csv("/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/radiomics_revised.csv")


In [70]:
result_df["eid"] = result_df["eid"].astype(int)

In [71]:
df_height = pd.read_csv(f"{PROJECT_DIR}/data/data/tabular/ukb668815_imaging.csv", usecols=["eid", "12144-2.0", "50-2.0"])

In [72]:
eids = result_df["eid"].to_list()
eids = [int(eid) for eid in eids]
df_height = df_height[df_height["eid"].isin(eids)]

In [73]:
# number of nan values in df_height
df_height.isna().sum()

eid            0
50-2.0       165
12144-2.0      0
dtype: int64

In [74]:
df_height["height"] = df_height["50-2.0"].fillna(df_height["12144-2.0"])

In [75]:
df_height = df_height.drop(columns=["50-2.0", "12144-2.0"])

In [76]:
# cm to m 
df_height["height"] = df_height["height"] / 100
df_height

eid  height
16     1001399    1.65
17     1001544    1.59
43     1004054    1.77
44     1004127    1.75
60     1005899    1.72
...        ...     ...
54197  6020110    1.64
54198  6020270    1.74
54203  6020602    1.66
54205  6020989    1.71
54235  6023419    1.71

[5762 rows x 2 columns]

In [77]:
result_df

spleen_shape_Elongation_wat  spleen_shape_Flatness_wat  \
0                        0.474130                   0.336946   
1                        0.742723                   0.384708   
2                        0.838894                   0.420246   
3                        0.722567                   0.437378   
4                        0.629149                   0.344354   
...                           ...                        ...   
5757                     0.706286                   0.383760   
5758                     0.681122                   0.362022   
5759                     0.671080                   0.387715   
5760                     0.536848                   0.417758   
5761                     0.597097                   0.397846   

      spleen_shape_LeastAxisLength_wat  spleen_shape_MajorAxisLength_wat  \
0                            17.143004                         50.877641   
1                            13.945688                         36.250014   
2                            15.700262                         37.359690   
3                            15.931646                         36.425322   
4                            14.520272                         42.166711   
...                                ...                               ...   
5757                         16.634087                         43.345027   
5758                         13.360568                         36.905374   
5759                         16.654334                         42.955093   
5760                         18.093834                         43.311709   
5761                         14.137402                         35.534894   

      spleen_shape_Maximum2DDiameterColumn_wat  \
0                                    50.000000   
1                                    40.718546   
2                                    42.059482   
3                                    36.400549   
4                                    44.654227   
...                                        ...   
5757                                 48.764741   
5758                                 39.446166   
5759                                 45.188494   
5760                                 36.235342   
5761                                 37.854986   

      spleen_shape_Maximum2DDiameterRow_wat  \
0                                 52.801515   
1                                 33.615473   
2                                 35.846897   
3                                 37.013511   
4                                 32.695565   
...                                     ...   
5757                              40.706265   
5758                              34.655447   
5759                              43.600459   
5760                              49.203658   
5761                              38.470768   

      spleen_shape_Maximum2DDiameterSlice_wat  \
0                                   28.844410   
1                                   30.886890   
2                                   39.115214   
3                                   30.232433   
4                                   32.449961   
...                                       ...   
5757                                36.674242   
5758                                28.844410   
5759                                32.202484   
5760                                28.442925   
5761                                25.942244   

      spleen_shape_Maximum3DDiameter_wat  spleen_shape_MeshVolume_wat  \
0                              54.963624                 12488.458333   
1                              41.158231                  8941.583333   
2                              42.813549                 11469.750000   
3                              38.131352                  7849.458333   
4                              45.541190                  9747.791667   
...                                  ...                          ...   
5757                           49.132474                 1

In [78]:
# merge radiomics and height
result_df = result_df.merge(df_height, how="left", on="eid")

In [79]:
# get all features with shape in name and remove all the prefix before shape
shape_features = [col for col in result_df.columns if "_shape_" in col]
shape_features = ["_".join(col.split("_shape_")[1:]) for col in shape_features]

In [80]:
shape_features = [col for col in shape_features if '_fat' in col]

In [81]:
# remove suffix _fat
shape_features = [col.split("_fat")[0] for col in shape_features]
shape_features = set(shape_features)

In [82]:
shape_features

{'Elongation',
 'Flatness',
 'LeastAxisLength',
 'MajorAxisLength',
 'Maximum2DDiameterColumn',
 'Maximum2DDiameterRow',
 'Maximum2DDiameterSlice',
 'Maximum3DDiameter',
 'MeshVolume',
 'MinorAxisLength',
 'Sphericity',
 'SurfaceArea',
 'SurfaceVolumeRatio',
 'VoxelVolume'}

In [83]:
# Assuming result_df is already defined and loaded with the necessary columns

# Define the list of feature names that require height squared normalization
features_to_normalize = [
    'LeastAxisLength', 'MajorAxisLength', 'Maximum2DDiameterColumn',
    'Maximum2DDiameterRow', 'Maximum2DDiameterSlice', 'Maximum3DDiameter',
    'MeshVolume', 'MinorAxisLength', 'SurfaceArea', 'SurfaceVolumeRatio', 'VoxelVolume'
]

labels_to_normalize = [
    "humerus_left",
    "humerus_right",
    "scapula_left",
    "scapula_right",
    "clavicula_left",
    "clavicula_right",
    "femur_left",
    "femur_right",
    "hip_left",
    "hip_right",
    "spinal_cord",
    "gluteus_maximus_left",
    "gluteus_maximus_right",
    "gluteus_medius_left",
    "gluteus_medius_right",
    "gluteus_minimus_left",
    "gluteus_minimus_right",
    "autochthon_left",
    "autochthon_right",
    "iliopsoas_left",
    "iliopsoas_right",
    "sternum",
    "costal_cartilages",
    "subcutaneous_fat",
    "muscle",
    "inner_fat",
    "IVD",
    "vertebra_body",
    "vertebra_posterior_elements",
    "spinal_channel",
    "bone_other"
]

# Normalize by height squared for each relevant 'shape' feature-label column
# Assumes there is a column 'height' in result_df
for feature in features_to_normalize:
    for label in labels_to_normalize:
        for suffix in ["fat", "wat"]:
            # Construct the column name based on label, feature, and suffix
            col_name = f"{label}_shape_{feature}_{suffix}"
            if col_name in result_df.columns:
                print(f"Normalizing {col_name}")
                result_df[col_name] = result_df[col_name] / (result_df['height'] ** 2)

    

Normalizing humerus_left_shape_LeastAxisLength_fat
Normalizing humerus_left_shape_LeastAxisLength_wat
Normalizing humerus_right_shape_LeastAxisLength_fat
Normalizing humerus_right_shape_LeastAxisLength_wat
Normalizing scapula_left_shape_LeastAxisLength_fat
Normalizing scapula_left_shape_LeastAxisLength_wat
Normalizing scapula_right_shape_LeastAxisLength_fat
Normalizing scapula_right_shape_LeastAxisLength_wat
Normalizing clavicula_left_shape_LeastAxisLength_fat
Normalizing clavicula_left_shape_LeastAxisLength_wat
Normalizing clavicula_right_shape_LeastAxisLength_fat
Normalizing clavicula_right_shape_LeastAxisLength_wat
Normalizing femur_left_shape_LeastAxisLength_fat
Normalizing femur_left_shape_LeastAxisLength_wat
Normalizing femur_right_shape_LeastAxisLength_fat
Normalizing femur_right_shape_LeastAxisLength_wat
Normalizing hip_left_shape_LeastAxisLength_fat
Normalizing hip_left_shape_LeastAxisLength_wat
Normalizing hip_right_shape_LeastAxisLength_fat
Normalizing hip_right_shape_LeastA

In [84]:
result_df.drop(columns=["height"], inplace=True)

In [85]:
result_df.to_csv("/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/radiomics_revised_norm_height_squared.csv", index=False)

## Preprocess and exclude prostate features and exceptions. Fuse fat and wat shape features into one

In [86]:
# drop columns with _exception_ in column name
result_df = result_df.drop(columns=[col for col in result_df.columns if "_exception_" in col])


In [87]:
result_df.to_csv("/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/radiomics_revised_norm_height_squared.csv", index=False)

In [92]:
result_df = pd.read_csv("/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/radiomics_revised_norm_height_squared.csv")

In [88]:
# remove prostate features
result_df = result_df.drop(columns=[col for col in result_df.columns if "prostate" in col])

In [89]:
result_df.to_csv("/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/radiomics_revised_norm_height_squared_no_prostate.csv", index=False)

In [90]:
result_df = pd.read_csv("/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/radiomics_revised_norm_height_squared_no_prostate.csv")

In [91]:
len(result_df.columns)

14981

In [93]:
# merge all shape _fat and _wat features into one in result_df
shape_features_wat = [col for col in result_df.columns if "_shape_" in col and col.endswith('_wat') ]
shape_features_fat = [col for col in result_df.columns if "_shape_" in col and col.endswith('_fat')]

In [94]:
len(shape_features_wat), len(shape_features_fat)

(994, 994)

In [95]:
result_df.drop(columns=shape_features_fat, inplace=True)

In [95]:
{col: col[:-4] for col in shape_features_wat}

{'spleen_shape_Elongation_wat': 'spleen_shape_Elongation',
 'spleen_shape_Flatness_wat': 'spleen_shape_Flatness',
 'spleen_shape_LeastAxisLength_wat': 'spleen_shape_LeastAxisLength',
 'spleen_shape_MajorAxisLength_wat': 'spleen_shape_MajorAxisLength',
 'spleen_shape_Maximum2DDiameterColumn_wat': 'spleen_shape_Maximum2DDiameterColumn',
 'spleen_shape_Maximum2DDiameterRow_wat': 'spleen_shape_Maximum2DDiameterRow',
 'spleen_shape_Maximum2DDiameterSlice_wat': 'spleen_shape_Maximum2DDiameterSlice',
 'spleen_shape_Maximum3DDiameter_wat': 'spleen_shape_Maximum3DDiameter',
 'spleen_shape_MeshVolume_wat': 'spleen_shape_MeshVolume',
 'spleen_shape_MinorAxisLength_wat': 'spleen_shape_MinorAxisLength',
 'spleen_shape_Sphericity_wat': 'spleen_shape_Sphericity',
 'spleen_shape_SurfaceArea_wat': 'spleen_shape_SurfaceArea',
 'spleen_shape_SurfaceVolumeRatio_wat': 'spleen_shape_SurfaceVolumeRatio',
 'spleen_shape_VoxelVolume_wat': 'spleen_shape_VoxelVolume',
 'kidney_right_shape_Elongation_wat': 'kidne

In [96]:
# remove _wat suffix from shape features in result_df
result_df = result_df.rename(columns={col: col[:-4] for col in shape_features_wat})

In [97]:
len(result_df.columns)

14201

In [98]:
len(result_df)

5762

In [99]:
result_df.to_csv("/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/radiomics_revised_norm_height_squared_no_prostate.csv", index=False)

## Get total_radiomics_tabular features for the specific dataset

In [99]:
eids_cohort_path = f"{PROJECT_DIR}/PrepareDataset/resources/3m_3y/cancer_3m_3y/"
eids_cohort = pd.read_csv(eids_cohort_path + "labels_with_val.csv")["eid"].to_list()

In [100]:
len(eids_cohort)

1202

In [104]:
result_df_cohort = result_df[result_df["eid"].isin(eids_cohort)]

In [105]:
len(result_df_cohort)

1194

In [107]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming result_df_cohort is already loaded
# Select all columns except 'eid' for processing
feature_columns = [col for col in result_df_cohort.columns if col != 'eid' and "prostate" not in col]

# Impute NaN values with the mean
result_df_cohort[feature_columns] = result_df_cohort[feature_columns].fillna(result_df_cohort[feature_columns].mean())

# Scale the features
scaler = StandardScaler()
result_df_cohort[feature_columns] = scaler.fit_transform(result_df_cohort[feature_columns])

/tmp/ipykernel_11524/2977787831.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df_cohort[feature_columns] = result_df_cohort[feature_columns].fillna(result_df_cohort[feature_columns].mean())
/tmp/ipykernel_11524/2977787831.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df_cohort[feature_columns] = scaler.fit_transform(result_df_cohort[feature_columns])


In [108]:
# scale prostate column with nans
prostate_columns = [col for col in result_df_cohort.columns if "prostate" in col]

scaler = StandardScaler()

for column in prostate_columns:
    non_nan_mask = ~result_df_cohort[column].isna()  # Mask for non-NaN values
    result_df_cohort.loc[non_nan_mask, column] = scaler.fit_transform(
        result_df_cohort.loc[non_nan_mask, column].values.reshape(-1, 1)
    ).flatten()

In [109]:
missing_values = result_df_cohort.isna().sum()
missing_values = missing_values.sort_values(ascending=False)
missing_values

prostate_ngtdm_Strength_fat                            459
prostate_gldm_HighGrayLevelEmphasis_wat                459
prostate_glrlm_GrayLevelVariance_wat                   459
prostate_glrlm_GrayLevelNonUniformityNormalized_wat    459
prostate_glrlm_GrayLevelNonUniformity_wat              459
                                                      ... 
clavicula_right_firstorder_TotalEnergy_wat               0
clavicula_right_firstorder_Uniformity_wat                0
clavicula_right_firstorder_Variance_wat                  0
clavicula_right_glcm_Autocorrelation_wat                 0
vertebra_body_glcm_DifferenceAverage_wat                 0
Length: 14201, dtype: int64

In [110]:
result_df_cohort.to_csv(eids_cohort_path + "/tabular_final_preprocessed/" + "total_radiomics_prostate_tabular.csv", index=False)

## Find Missing eids that should be extracted

In [39]:
DATASETS = ["pancreas_3m_3y", "liver_3m_3y", "cancer_3m_3y", "copd_3m_3y", "ckd_3m_3y", "cvd2_3m_3y", "osteoarthritis_3m_3y"]
eids_all = []
for dataset in DATASETS:
    cohort_path = f'{PROJECT_DIR}/PrepareDataset/resources/3m_3y/{dataset}/'
    eids_cohort = pd.read_csv(cohort_path + "labels_with_val.csv")["eid"].to_list()
    eids_all.extend(eids_cohort)
eids_all = list(set(eids_all))

In [40]:
len(eids_all)

5771

In [57]:
pd.DataFrame(eids_all, columns=["eid"]).to_csv(f"{PROJECT_DIR}/PrepareDataset/resources/3m_3y/eids_all.csv", index=False)

In [41]:
nifti_path = f"{PROJECT_DIR}/data/data/whole_body/nifti"
eids_with_nifti = []
for eid in eids_all:
    if os.path.exists(os.path.join(nifti_path, str(eid))):
        eids_with_nifti.append(eid)

KeyboardInterrupt: 

In [58]:
total_segmentator_path = f"{PROJECT_DIR}/data/data/whole_body/total_segmentator"
eids_missing = []
for eid in eids_with_nifti:
    if not os.path.exists(os.path.join(total_segmentator_path, str(eid))):
        eids_missing.append(eid)

In [59]:
len(eids_missing)

2071

In [60]:
len(eids_with_nifti)

5764

In [61]:
# save eids with nifti
pd.DataFrame(list(eids_missing), columns=["eid"]).to_csv(f"{PROJECT_DIR}/PrepareDataset/resources/3m_3y/missing_eids.csv", index=False)

In [62]:
# divide eids with nifti into 5 equal parts
for i in range(5):
    if i == 4:
        eids_part = eids_missing[i*int(len(eids_missing)/5):]
    else:
        eids_part = eids_missing[i*int(len(eids_missing)/5):(i+1)*int(len(eids_missing)/5)]
    pd.DataFrame(eids_part, columns=["eid"]).to_csv(f"{PROJECT_DIR}/PrepareDataset/resources/3m_3y/missing_eids_{i}.csv", index=False)


## Visualize Results


In [143]:
import pandas as pd
data_path = (f"{PROJECT_DIR}/analysisNumericFeatures/tabular_experiment/results/results_cvfull/2024-11-28 17:21:15/cvd2_3m_3y/non_image+cardiac/summary.csv")
df = pd.read_csv(data_path)

def format_mean_std(mean, std):
    return f"{mean:.3f} ± {std:.3f}"
# Create the DataFrame with the updated column names

# Combine mean and std columns for each metric
metrics = ["balanced_accuracy_mean", "f1_mean", "roc_auc_mean"]
for metric in metrics:
    df[metric] = df.apply(lambda x: format_mean_std(x[f"{metric}_mean"], x[f"{metric}_std"]), axis=1)

# Keep only the combined columns
df = df[["model_"] + metrics]

df

model_ balanced_accuracy_mean        f1_mean   roc_auc_mean
0  CatBoost          0.613 ± 0.005  0.599 ± 0.006  0.656 ± 0.004
1       MLP          0.592 ± 0.007  0.570 ± 0.028  0.634 ± 0.008
2        RF          0.613 ± 0.008  0.597 ± 0.008  0.656 ± 0.005
3       XGB          0.611 ± 0.007  0.599 ± 0.009  0.657 ± 0.005

In [58]:
extracted_radiomics_path = "/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/radiomics_revised/"
extracted_radiomics_files = os.listdir(extracted_radiomics_path)

In [45]:
extracted_total_seg = "/home/dmitrii/data/sdm_files/Projects/TotalVibeSegmentator/total_segmentator/"
extracted_total_seg_files = os.listdir(extracted_total_seg)

In [44]:
eids_missing = pd.read_csv(f"{PROJECT_DIR}/PrepareDataset/resources/3m_3y/missing_eids.csv")["eid"].to_list()

In [61]:
extracted_total_seg_files = [int(file) for file in extracted_total_seg_files]
extracted_radiomics_files = [int(file) for file in extracted_radiomics_files]

In [62]:
eids_to_extract = list(set(eids_cohort) - set(extracted_radiomics_files))

In [63]:
len(eids_to_extract)

0

In [57]:
pd.DataFrame(eids_to_extract, columns=["eid"]).to_csv(f"{PROJECT_DIR}/data/projects/risk_assessment/labels/3m_3y/eids_to_extract_osteoarthritis.csv", index=False)

In [80]:
import pandas as pd
import os
import numpy as np
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import balanced_accuracy_score, f1_score, roc_auc_score
import json

exp_dir = f"{PROJECT_DIR}/analysisNumericFeatures/tabular_experiment/results/results_val/2024-11-18 15:14:25/pancreas_3m_3y/non_image+total_segmentator/validation"

labels = f"{PROJECT_DIR}/PrepareDataset/resources/3m_3y/pancreas_3m_3y/labels_with_val.csv"
data = f"{PROJECT_DIR}/PrepareDataset/resources/3m_3y/pancreas_3m_3y/tabular_final_preprocessed/total_radiomics_mrmr_20.csv"
labels = pd.read_csv(labels)
data = pd.read_csv(data)
data = data.merge(labels, how="left", on="eid") 
# based on "split in labels create train and val sets
train_data = data[data["split"] == "train"]
train_data.drop(columns=["eid", "split", "time_to_event"], inplace=True)
test_data = data[data["split"] == "test"]
test_data.drop(columns=["eid", "split", "time_to_event"], inplace=True)
model_dict = {
    "CatBoost": CatBoostClassifier,
    "RF": RandomForestClassifier,
    "XGB": XGBClassifier,
    "MLP": MLPClassifier
}
best_params_default = {
    "CatBoost": {
        "bootstrap_type": "Bayesian",
        "eval_metric": "BalancedAccuracy",
        "verbose": 0
    },
    "RF": {
        "bootstrap": True,
        "oob_score": True,
    },
    "XGB": {
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "tree_mathod": "exact"
    },
    "MLP": {
        "activation": "relu",
        "early_stopping": True,
        "validation_fraction": 0.1,
        "n_iter_no_change": 10,
        "solver": "adam",
        "max_iter": 10000
    }
}
results_all = {}

for model_name, model in model_dict.items():
    print("Model: ", model_name)
    results = {
        "balanced_accuracy": [],
        "f1": [],
        "roc_auc": []
    }
    for seed in [1514, 42, 0, 29847, 867228]:
        # read the best param json file
        best_param_path = os.path.join(exp_dir,model_name, f"best_params_seed_{str(seed)}.json")
        with open(best_param_path, "r") as f:
            best_params = json.load(f)
        
        for k, v in best_params_default[model_name].items():
            best_params[k] = v
        
        model_trained = model(random_state=seed, **best_params)
        model_trained.fit(train_data.drop(columns=["event"]), train_data["event"])
        preds = model_trained.predict(test_data.drop(columns=["event"]))
        preds_proba = model_trained.predict_proba(test_data.drop(columns=["event"]))[:, 1]
        balanced_accuracy = balanced_accuracy_score(test_data["event"], preds)
        f1 = f1_score(test_data["event"], preds)
        roc_auc = roc_auc_score(test_data["event"], preds_proba)
        
        results["balanced_accuracy"].append(balanced_accuracy)
        results["f1"].append(f1)
        results["roc_auc"].append(roc_auc)
        results["y_pred"] = preds
        results["y_proba"] = preds_proba
    results_all[model_name] = results
    

for model_name, results in results_all.items():
    for metric, values in results.items():
        results_all[model_name][metric] = f"{np.mean(values):.3f} ± {np.std(values):.3f}"
        
        
    

/tmp/ipykernel_11524/4104748206.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop(columns=["eid", "split", "time_to_event"], inplace=True)
/tmp/ipykernel_11524/4104748206.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(columns=["eid", "split", "time_to_event"], inplace=True)


Model:  CatBoost
Model:  RF
Model:  XGB


/home/dmitrii/miniconda3/envs/ukbb/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:15:47] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "tree_mathod" } are not used.

  warnings.warn(smsg, UserWarning)
/home/dmitrii/miniconda3/envs/ukbb/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:15:48] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "tree_mathod" } are not used.

  warnings.warn(smsg, UserWarning)
/home/dmitrii/miniconda3/envs/ukbb/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:15:53] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "tree_mathod" } are not used.

  warnings.warn(smsg, UserWarning)
/home/dmitrii/miniconda3/envs/ukbb/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:15:54] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "tree_mathod" } are not used.

  warnings.warn(smsg, UserWarning)
/home/dmitrii/miniconda3/envs/ukbb/lib/python3.10/site-packages/xgboost/core.py:

Model:  MLP


In [81]:
results_all

{'CatBoost': {'balanced_accuracy': '0.719 ± 0.022',
  'f1': '0.707 ± 0.019',
  'roc_auc': '0.771 ± 0.023',
  'y_pred': '0.449 ± 0.497',
  'y_proba': '0.496 ± 0.237'},
 'RF': {'balanced_accuracy': '0.708 ± 0.018',
  'f1': '0.712 ± 0.020',
  'roc_auc': '0.768 ± 0.005',
  'y_pred': '0.508 ± 0.500',
  'y_proba': '0.510 ± 0.168'},
 'XGB': {'balanced_accuracy': '0.736 ± 0.020',
  'f1': '0.721 ± 0.022',
  'roc_auc': '0.791 ± 0.010',
  'y_pred': '0.449 ± 0.497',
  'y_proba': '0.499 ± 0.220'},
 'MLP': {'balanced_accuracy': '0.664 ± 0.043',
  'f1': '0.683 ± 0.046',
  'roc_auc': '0.731 ± 0.028',
  'y_pred': '0.814 ± 0.389',
  'y_proba': '0.718 ± 0.228'}}

In [59]:
results_all

{'CatBoost': {'balanced_accuracy': '0.568 ± 0.005',
  'f1': '0.536 ± 0.007',
  'roc_auc': '0.574 ± 0.004'},
 'RF': {'balanced_accuracy': '0.577 ± 0.012',
  'f1': '0.540 ± 0.012',
  'roc_auc': '0.588 ± 0.013'}}

In [85]:
df_data = {"eid": [1,2, 3, 4, 5], "event": [0, 1, 1, 1, 0], "split": ["test", "train", "test", "test", "train"], "time_to_event": [1, 2, 3, 4, 5]}
df = pd.DataFrame(df_data)
df

eid  event  split  time_to_event
0    1      0   test              1
1    2      1  train              2
2    3      1   test              3
3    4      1   test              4
4    5      0  train              5

In [87]:
df[df["split"] == "train"]["eid"]

1    2
4    5
Name: eid, dtype: int64

In [69]:
#dataset_path = "{PROJECT_DIR}/data/projects/risk_assessment/labels/3m_3y"
dataset_path = f"{PROJECT_DIR}/PrepareDataset/resources/3m_3y"
datasets = ["cvd2_3m_3y", "pancreas_3m_3y", "liver_3m_3y","cancer_3m_3y", "copd_3m_3y", "ckd_3m_3y", "osteoarthritis_3m_3y"]
for dataset in datasets:
    print(dataset)
    total_radiomics = pd.read_csv(f"{dataset_path}/{dataset}/tabular_final_preprocessed/total_radiomics_mrmr_20.csv")
    non_image = pd.read_csv(f"{dataset_path}/{dataset}/tabular_final_preprocessed/nonimage_tabular.csv")
    df = total_radiomics.merge(non_image, how="inner", on="eid")
    df.to_csv(f"{dataset_path}/{dataset}/tabular_final_preprocessed/nonimage_total_mrmr_20.csv", index=False)
    print(len(total_radiomics), len(non_image), len(df))

cvd2_3m_3y
1582 1584 1582
pancreas_3m_3y
590 590 590
liver_3m_3y
406 406 406
cancer_3m_3y
1194 1202 1194
copd_3m_3y
386 385 385
ckd_3m_3y
608 607 607
osteoarthritis_3m_3y
1840 1840 1840
